# Senior Design Team 1725, Classifying AIS, MM Solution Training
This file is a jupyter notebook containing
the training results for the "multiple models"
solution for the AIS Classification project.

Intent: showcase a training method that can work
        with any dataset size for a specific class
        while also being flexible for training new models
        that can handle other invasive species.

Basically, rather than having one model that handles all species,
we attempt to have multiple models chained together to handle identification

##Training Run: 5
This is the fifth version of the file. This time, we try to accomplish a more advanced training targeting purple loosestrife.
- From the purple loosestrife folder, we use ~1600 images (entire folder).
- From the none folder, we take ~200  images (entire folder)
- From the narrowleaf cattail folder, we use 700 images
- From the phragmites folder, we use 700 images
- Dataset split: 80% training, 10% validation, 10% testing
- For our pre-trained model, we use YOLOv11n-cls
- For our hyperparameters, we use: 75 epochs, 32 batch size
- We are using an A100 GPU instance

Additionally, we attempt mounting our dataset via Google Drive in order to test if this method saves time. A zip file containing the dataset is uploaded to drive, and, when Drive is mounted, we attempt to download to the Colab instance.

## Dataset Mounting (Sharepoint):
First, we want to access our dataset. To do this, we mount our sharepoint/OneDrive to colab. Can be done by following [this](https://medium.com/@pratikmukherjee32/mount-onedrive-to-google-colab-f941a3a96a79) guide.

You must first run the rclone config *locally*, then you can get it on Colab. The reason for this is because we need an API key for the mount to work properly, which requires web browser access (not possible in Colab).

In [ ]:
!wget https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
!apt install ./rclone-v1.69.0-linux-amd64.deb

--2025-02-14 00:00:02--  https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23665802 (23M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.69.0-linux-amd64.deb’

rclone-v1.69.0-linu 100%[===================>]  22.57M  14.0MB/s    in 1.6s    

2025-02-14 00:00:04 (14.0 MB/s) - ‘rclone-v1.69.0-linux-amd64.deb’ saved [23665802/23665802]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.69.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 0 B/23.7 MB of archives.
After this operation, 71.0 MB of additional disk space will be used.
Get

In [ ]:
# We will need to pass some arguments for this to work,
# so make sure we still have the api key
# (Note: you need to copy the ENTIRE struct, as the values are read as an array)

# Also, make sure to clear the output! We don't want to have
# loose api keys floating around the internet!
!rclone config

In [ ]:
!sudo mkdir /content/onedrive
# Create onedrive directory in colab

In [ ]:
# Install fusermount3 to ensure proper mounting
!sudo apt-get -y install fuse3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libfuse3-3
The following packages will be REMOVED:
  fuse
The following NEW packages will be installed:
  fuse3 libfuse3-3
0 upgraded, 2 newly installed, 1 to remove and 20 not upgraded.
Need to get 106 kB of archives.
After this operation, 264 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfuse3-3 amd64 3.10.5-1build1 [81.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fuse3 amd64 3.10.5-1build1 [24.7 kB]
Fetched 106 kB in 1s (129 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 

In [ ]:
!nohup rclone --vfs-cache-mode full mount onedrive: /content/onedrive &
# After these commands, we should have a onedrive folder in the files section
# within colab with everything we need!

nohup: appending output to 'nohup.out'


## Dataset Mounting (Google Drive):
Alternative method to access dataset. Here, we simply mount Google Drive to colab, then unzip the contents of the zip file to the colab instance

EDIT: The zip file to Drive was too big, and wouldn't copy over properly for processing. Thus, we are going to continue using the onedrive sharepoint.  This issue will be documented inside the weekly report for 2/14/2025

In [ ]:
# prompt: mount google drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# prompt: from drive, access SenDsgn folder, unzip contents of "Images.zip" to directory in colab called "SenDsgnTraining"

import zipfile
import os

# Create the SenDsgnTraining directory if it doesn't exist
if not os.path.exists('/content/SenDsgnTraining/Images'):
    os.makedirs('/content/SenDsgnTraining/Images')

# Specify the path to the zip file in your Google Drive
zip_file_path = '/content/drive/MyDrive/SenDsgn/Images.zip'

# Extract the contents of the zip file to the SenDsgnTraining directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/SenDsgnTraining')


OSError: [Errno 28] No space left on device

## Dataset Setup:
What we want to do next is copy the files from the datasets we want to use for creating our first few models.

Basically, we are copying the files to the colab environment because we don't want to overwrite what's going on with OneDrive and because we want a "fresh" place to store our training results

##Word of warning:
Doing this will consume disk resources on Colab, make sure you have enough space with the images you copy over!

In [ ]:
# Import utilities to take random image sampling and copy to target directory
import os
import shutil
import random

# Create a new directory specifically to hold dataset
!sudo mkdir /content/SenDsgnTraining

In [ ]:
# Function made to help with splitting datasets. Run before you attempt
# to make any calls!
def distributeData(sd, td, n, cls, tgt, splits):
  # sd = source_dir, td = target_dir
  # n = number of images, cls = image class, tgt = is targeted class

  # Processes images
  source_path = os.path.join(sd, cls)
  images = [f for f in os.listdir(source_path)]

  # Randomly sample images
  selected_images = random.sample(images, min(n, len(images)))

  # Distribute images across train, val, test
  split_counts = {k: int(v * len(selected_images) / 100) for k, v in splits.items()}
  start_idx = 0

  for split, count in split_counts.items():
    # Ensure split_path goes to either "none" or the targeted class
    split_path = os.path.join(td, split)
    if tgt == cls:
        split_path = os.path.join(split_path, cls)
    else:
        split_path = os.path.join(split_path, "none")

    for i in range(count):
        # Grab file extension, create image name path, then copy
        file_ext = os.path.splitext(selected_images[start_idx])[1]
        img_name = f"{start_idx}_{cls}{file_ext}"
        shutil.copy(os.path.join(source_path, selected_images[start_idx]), os.path.join(split_path, img_name))
        start_idx += 1

In [ ]:
# Dataset path (edited to use drive paths):
source_dir = r"/content/onedrive/Training Images/Training"
target_dir = r"/content/SenDsgnTraining"

# Set targeted class. This will be used specifically to ensure proper function
# for the 2-class setup
tgt_cls = "purple_loosestrife"
splits = {"train": 80, "val": 10, "test": 10}  # Percentage split

# Create target directory structure
for split in splits:
    os.makedirs(os.path.join(target_dir, split, tgt_cls), exist_ok=True)
    os.makedirs(os.path.join(target_dir, split, "none"), exist_ok=True)

In [ ]:
# Process each class individually
# Start with narrowleaf cattail
n = 700
distributeData(source_dir, target_dir, n, "narrowleaf_cattail", tgt_cls, splits)

In [ ]:
# Now, onto the "none" class
n = 200
distributeData(source_dir, target_dir, n, "none", tgt_cls, splits)

In [ ]:
# Phragmites
n = 700
distributeData(source_dir, target_dir, n, "phragmites", tgt_cls, splits)

In [ ]:
# Purple Loosestrife
n = 1600
distributeData(source_dir, target_dir, n, "purple_loosestrife", tgt_cls, splits)

# Hopefully, the dataset should now be appropriately split
# into the target and none classes!

In [ ]:
# NEXT TIME: DO NOT RUN THIS BLOCK!
# Rename all files in the "none" splits
# This is to ensure the model properly reads all the images instead of the ones with "none"

# prompt: Rename all files in the "none" splits using the format: img_name = f"{idx}_none.jpg"
target_dir = r"/content/SenDsgnTraining"

for split in ["train", "val", "test"]:
  none_dir = os.path.join(target_dir, split, "none")
  for idx, filename in enumerate(os.listdir(none_dir)):
    if not filename.startswith(f"{idx}_none"):  # Check if already renamed
        source_path = os.path.join(none_dir, filename)
        new_filename = f"{idx}_none.jpg"
        destination_path = os.path.join(none_dir, new_filename)
        os.rename(source_path, destination_path)

# Hopefully, the dataset should now be appropriately split
# into the target and none classes!

## Training
Now that we've gotten our dataset mounted and our splits taken care of, we can begin training.

The best thing about the classification model is that no .yaml file is necessary, as the folder structure is enough to ensure the training, validation, and testing splits are recognized for each class!

In [ ]:
# Install Ultralytics and dependencies
%pip install ultralytics  # install
from ultralytics import YOLO, checks
checks()  # checks

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 94.1/235.7 GB disk)


In [ ]:
# Load a model
model = YOLO("yolo11n-cls.pt")  # Load a pretrained model

# Train the model
results = model.train(data=target_dir, epochs=75, imgsz=640, batch=32)

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/SenDsgnTraining, epochs=75, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning /content/SenDsgnTraining/train... 2333 images, 0 corrupt: 100%|██████████| 2333/2333 [00:04<00:00, 484.12it/s] 

train: New cache created: /content/SenDsgnTraining/train.cache



val: Scanning /content/SenDsgnTraining/val... 310 images, 0 corrupt: 100%|██████████| 310/310 [00:00<00:00, 442.38it/s]

val: New cache created: /content/SenDsgnTraining/val.cache


...

      75/75      3.62G  0.0008513         29        640: 100%|██████████| 73/73 [02:00<00:00,  1.65s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00, 23.04it/s]

                   all          1          1



75 epochs completed in 2.608 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 3.2MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n-cls summary (fused): 112 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /content/SenDsgnTraining/train... found 2333 images in 2 classes ✅ 
val: /content/SenDsgnTraining/val... found 310 images in 2 classes ✅ 
test: /content/SenDsgnTraining/test... found 310 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 5/5 [00:00<00:00,  6.05it/s]


                   all          1          1
Speed: 1.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [ ]:
# Just in case, we want to download the "runs" folder and save our training results
!zip -r /content/run5.zip /content/runs

updating: content/runs/ (stored 0%)
updating: content/runs/classify/ (stored 0%)
updating: content/runs/classify/train/ (stored 0%)
updating: content/runs/classify/train/events.out.tfevents.1739493476.8e5c565b346e.3750.0 (deflated 9%)
updating: content/runs/classify/train/args.yaml (deflated 52%)
updating: content/runs/classify/train/weights/ (stored 0%)
updating: content/runs/classify/train2/ (stored 0%)
updating: content/runs/classify/train2/train_batch1.jpg (deflated 1%)
updating: content/runs/classify/train2/val_batch1_pred.jpg (deflated 0%)
updating: content/runs/classify/train2/results.png (deflated 12%)
updating: content/runs/classify/train2/train_batch4745.jpg (deflated 1%)
updating: content/runs/classify/train2/confusion_matrix.png (deflated 33%)
updating: content/runs/classify/train2/train_batch2.jpg (deflated 1%)
updating: content/runs/classify/train2/train_batch4746.jpg (deflated 1%)
updating: content/runs/classify/train2/args.yaml (deflated 52%)
updating: content/runs/clas

In [ ]:
results2 = model.val()

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n-cls summary (fused): 112 layers, 1,528,586 parameters, 0 gradients, 3.2 GFLOPs
train: /content/SenDsgnTraining/train... found 2333 images in 2 classes ✅ 
val: /content/SenDsgnTraining/val... found 310 images in 2 classes ✅ 
test: /content/SenDsgnTraining/test... found 310 images in 2 classes ✅ 


val: Scanning /content/SenDsgnTraining/val... 310 images, 0 corrupt: 100%|██████████| 310/310 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 10/10 [00:25<00:00,  2.50s/it]


                   all          1          1
Speed: 5.0ms preprocess, 1.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train22
